In [ ]:
# Imports
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from timeit import default_timer as timer

In [ ]:
# Parameters 
# DATASET_FILE_PATH: "BigData.xlsx",
# DATASET_SHEET_TITLE: "data_test",
# GRANULARITY: 10, # Take first item, skip next x, take next, skip x, take next, ...
# STEP_SIZE_SLIDING_WINDOW: 5,
# PAST_HISTORY: 20,
# FUTURE_TARGET: 1, 
# Looks at observations that are PASTHISTORY timesteps prior (reading only every STEP_SIZE_SLIDING_WINDOW entry),
# then predicts the result at FUTURETARGET timesteps in the future
# VAL_PERCENT: 0.1,
# TEST_PERCENT: 0.1,
# EPOCHS: 5,
# BATCH_SIZE: 10 # Splits the dataset into batches of this size: we perform gradiant descent once per batch
# SMOOTHING: 0

# runAll(DATASET_FILE_PATH, DATASET_SHEET_TITLE, GRANULARITY, STEP_SIZE_SLIDING_WINDOW, PAST_HISTORY, 
#        FUTURE_TARGET, VAL_PERCENT, TEST_PERCENT, EPOCHS, BATCH_SPLITS_TRAIN, BATCH_SPLITS_VAL, SMOOTHING)

In [ ]:
def run_all(DATASET_FILE_PATH, DATASET_SHEET_TITLE, GRANULARITY, STEP_SIZE_SLIDING_WINDOW, PAST_HISTORY, 
            FUTURE_TARGET, VAL_PERCENT, TEST_PERCENT, EPOCHS, BATCH_SPLITS, SMOOTHING, ATTEMPT_NAME, 
            SHUFFLE_BUFFER_SIZE, MEAN): 
    indexes, ground_truth, x_train, x_val, batched_train_data, batched_val_data, batched_test_data = process_data(
        DATASET_FILE_PATH, DATASET_SHEET_TITLE, GRANULARITY, SMOOTHING, ATTEMPT_NAME, VAL_PERCENT, TEST_PERCENT, 
        PAST_HISTORY, FUTURE_TARGET, STEP_SIZE_SLIDING_WINDOW, BATCH_SPLITS, EPOCHS, SHUFFLE_BUFFER_SIZE, MEAN)
    model, training_history, training_time = run_lstm(x_train, x_val, batched_train_data, batched_val_data, 
                                                      BATCH_SPLITS, EPOCHS, FUTURE_TARGET) 
    evaluate_results(model, training_history, indexes, ground_truth, batched_test_data, training_time)

In [ ]:
# Load algorithm
%run ./data_management_component.ipynb

In [ ]:
%run ./lstm_baseline.ipynb

In [ ]:
%run ./evaluation_component.ipynb

In [ ]:
DATASET_FILE_PATH="Datasets\\BigData.xlsx"
DATASET_SHEET_TITLE="data_test"
GRANULARITY=10
STEP_SIZE_SLIDING_WINDOW=5
PAST_HISTORY=20
FUTURE_TARGET=1
VAL_PERCENT=0.1
TEST_PERCENT=0.1
EPOCHS=5
BATCH_SPLITS=10
SMOOTHING=50
ATTEMPT_NAME="LSTM_BASELINE"
SHUFFLE_BUFFER_SIZE=100
MEAN=True

# Running the algorithm all at once
BASELINE = run_all(DATASET_FILE_PATH, DATASET_SHEET_TITLE, GRANULARITY, STEP_SIZE_SLIDING_WINDOW, 
                   PAST_HISTORY, FUTURE_TARGET, VAL_PERCENT, TEST_PERCENT, EPOCHS, BATCH_SPLITS, 
                   SMOOTHING, ATTEMPT_NAME, SHUFFLE_BUFFER_SIZE, MEAN)